In [58]:
from youtube_transcript_api import YouTubeTranscriptApi, TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate

**Step 1a - Indexing (Document Ingestion)**

In [ ]:
video_id = "ubOo3NEA4rA"
try:
    transcript_list = YouTubeTranscriptApi().fetch(video_id, languages=['hi'])
    # transcript = " ".join(chunk["text"] for chunk in transcript_list)
    print(transcript_list.snippets)
except TranscriptsDisabled:
    print("No Captions Available for this video.")

[FetchedTranscriptSnippet(text="here's a fun fact the average lifespan", start=0.04, duration=4.2), FetchedTranscriptSnippet(text='of a Creator is roughly 3 months yes', start=1.839, duration=4.641), FetchedTranscriptSnippet(text="that's right about 100 days is as long", start=4.24, duration=4.08), FetchedTranscriptSnippet(text='as the average influencer will last so', start=6.48, duration=3.159), FetchedTranscriptSnippet(text='at some point they have to leave their', start=8.32, duration=3.8), FetchedTranscriptSnippet(text='platform be it YouTube Tik Tok Instagram', start=9.639, duration=4.321), FetchedTranscriptSnippet(text='Facebook and go back to the real world', start=12.12, duration=3.36), FetchedTranscriptSnippet(text="they'll have to apply for a job cuz", start=13.96, duration=2.88), FetchedTranscriptSnippet(text='being a YouTuber is an absolute', start=15.48, duration=2.719), FetchedTranscriptSnippet(text='privilege it is the best job in the', start=16.84, duration=3.04), Fetc

In [33]:
import json

# Method 1: Convert to list of dictionaries
transcript = []
for snippet in transcript_list:
    transcript.append({
        "text": snippet.text,
        "start": snippet.start,
        "duration": snippet.duration
    })

print(json.dumps(transcript, indent=2))

[
  {
    "text": "here's a fun fact the average lifespan",
    "start": 0.04,
    "duration": 4.2
  },
  {
    "text": "of a Creator is roughly 3 months yes",
    "start": 1.839,
    "duration": 4.641
  },
  {
    "text": "that's right about 100 days is as long",
    "start": 4.24,
    "duration": 4.08
  },
  {
    "text": "as the average influencer will last so",
    "start": 6.48,
    "duration": 3.159
  },
  {
    "text": "at some point they have to leave their",
    "start": 8.32,
    "duration": 3.8
  },
  {
    "text": "platform be it YouTube Tik Tok Instagram",
    "start": 9.639,
    "duration": 4.321
  },
  {
    "text": "Facebook and go back to the real world",
    "start": 12.12,
    "duration": 3.36
  },
  {
    "text": "they'll have to apply for a job cuz",
    "start": 13.96,
    "duration": 2.88
  },
  {
    "text": "being a YouTuber is an absolute",
    "start": 15.48,
    "duration": 2.719
  },
  {
    "text": "privilege it is the best job in the",
    "start": 16.84,

**Step 1b - Indexing (Text Splitting)**

In [36]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
full_text = " ".join([t['text'] for t in transcript])
chunks = splitter.create_documents([full_text])

In [39]:
len(chunks)

33

In [37]:
chunks[0]

Document(metadata={}, page_content="here's a fun fact the average lifespan of a Creator is roughly 3 months yes that's right about 100 days is as long as the average influencer will last so at some point they have to leave their platform be it YouTube Tik Tok Instagram Facebook and go back to the real world they'll have to apply for a job cuz being a YouTuber is an absolute privilege it is the best job in the world and I do not take it for granted any single day but there will be a day inevitably when I do stop YouTube and when that day comes for most people they're going to have to return to the real world and stop living in this fantasy world of being a YouTuber and so that had me thinking did I actually learn anything from being a YouTuber do I have a resume that I can then show off and bring to my next job am I hirable well today we're going to figure that out because I will be applying to multiple jobs as a YouTuber what does that mean well I'm going to be discounting everything I

**Step 1c & 1d - Indexing (Embedding Generation and Storing in Vector Store)**

In [53]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embeddings)

In [54]:
vectorstore.index_to_docstore_id

{0: '10803f55-64d5-48dc-b47a-6d7d5a8a9f25',
 1: '548546bc-5029-4574-a4a4-08cd1d9be99f',
 2: 'e8a4faf5-795a-456c-b67b-0cf78096a094',
 3: '24e10376-6355-41fe-9aa7-d593c2802352',
 4: '0ecc9257-b7e9-4ad3-95f0-e9b5f7195288',
 5: '3d8cc679-1ac7-46fc-aff1-79c8f6f8d62e',
 6: 'cc6756c0-521f-4b5d-a15f-72c91085710e',
 7: '9e3c8a8c-daca-4928-822f-1b1b08c0e0fb',
 8: 'c3d2a72e-2b32-4f9b-bb9f-94fe86b8ec56',
 9: 'e403bf28-e489-4547-b4e7-7346bbc44a28',
 10: '053f8081-0401-48ca-a9e6-2b578dab1a81',
 11: 'b8e2a1b9-ac67-4604-b4a0-193169903644',
 12: '8d155ef0-82ee-4d41-92f2-a738f1f673a4',
 13: '9c050013-59a8-410a-bb4b-7745d013495b',
 14: 'f96f6987-260c-48e9-8781-0cca799620d1',
 15: '0e0d5ed2-cee1-48d2-8900-c0f1f86f7e5c',
 16: '8a8d2505-f783-4137-b402-3b3cc55792db',
 17: '73c58040-7740-4208-be8a-46d7ffd738ef',
 18: '2c2985fa-dabf-4ecb-95ec-12174fec5b64',
 19: 'e7b55563-030a-4753-81f3-f2844ad5e504',
 20: 'b8cd27ea-df83-460a-bdea-02a7c6a7f21c',
 21: '36981076-83b8-400f-963f-b28b76d61e27',
 22: 'ac80cfc3-4bbe-

**Step 2 - Retrieval**

In [55]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [57]:
retriever.invoke('What is Lifespan of a Content Creator')

[Document(id='10803f55-64d5-48dc-b47a-6d7d5a8a9f25', metadata={}, page_content="here's a fun fact the average lifespan of a Creator is roughly 3 months yes that's right about 100 days is as long as the average influencer will last so at some point they have to leave their platform be it YouTube Tik Tok Instagram Facebook and go back to the real world they'll have to apply for a job cuz being a YouTuber is an absolute privilege it is the best job in the world and I do not take it for granted any single day but there will be a day inevitably when I do stop YouTube and when that day comes for most people they're going to have to return to the real world and stop living in this fantasy world of being a YouTuber and so that had me thinking did I actually learn anything from being a YouTuber do I have a resume that I can then show off and bring to my next job am I hirable well today we're going to figure that out because I will be applying to multiple jobs as a YouTuber what does that mean w

**Step 3 - Augmentation**

In [59]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.2)

In [60]:
prompt = PromptTemplate(
    template="""
        You are a helpful AI assistant.
        Answer Only from the provided transcript context.
        If the context is Insufficient, just say you don't know.

        {context}
        Question: {question}
    """,
    input_variables=["context", "question"]
)

In [63]:
question = "What is Lifespan of a Content Creator"
retrieved_docs = retriever.invoke(question)

In [64]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)

In [65]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

**Step 4 - Generation**

In [67]:
answer = llm.invoke(final_prompt)
print(answer.content)

The average lifespan of a Creator is roughly 3 months, which is about 100 days.


**Building a Chain**

In [68]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [69]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [70]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})

In [71]:
parallel_chain.invoke("What is Lifespan of a Content Creator")

{'context': "here's a fun fact the average lifespan of a Creator is roughly 3 months yes that's right about 100 days is as long as the average influencer will last so at some point they have to leave their platform be it YouTube Tik Tok Instagram Facebook and go back to the real world they'll have to apply for a job cuz being a YouTuber is an absolute privilege it is the best job in the world and I do not take it for granted any single day but there will be a day inevitably when I do stop YouTube and when that day comes for most people they're going to have to return to the real world and stop living in this fantasy world of being a YouTuber and so that had me thinking did I actually learn anything from being a YouTuber do I have a resume that I can then show off and bring to my next job am I hirable well today we're going to figure that out because I will be applying to multiple jobs as a YouTuber what does that mean well I'm going to be discounting everything I've ever done in the pa

In [72]:
parser = StrOutputParser()

In [73]:
main_chain = parallel_chain | prompt | llm | parser

In [ ]:
main_chain.invoke("Summarize the video")

'The video appears to be about the speaker\'s experience applying for a job, despite having a "poor resume" and lacking "qualifying skills." They discuss their YouTube career, mentioning a difficult video made with a chimpanzee in Florida and expressing that YouTube is no longer fulfilling, causing "brain rot" from creating "dumb concepts," and they wish to step back into the "real world." The interviewer acknowledged their creativity and sizable YouTube audience. The speaker was surprised by the outcome of the job application, as they expected to be denied but the interviewer seemed interested in leveraging their YouTube audience. The video concludes with the speaker feeling "a little bit upset" about the situation and thanking their viewers.'

: 